# Fine Tune
- RoBERTa
- No need for inference speed up using distil bert since dataset is very small
- Hyperparameter tuning using huggingfaces hyperparameter search
- group k fold cross validation for prediction

## Several conditions:
- (spell corrected and) expanded prompts
- raw conversational part


In [2]:
import torch
print(torch.backends.mps.is_available())
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

True


In [3]:
import sqlite3
import pandas as pd

conn  = sqlite3.connect('../../../data/giicg.db')
all_prompts = pd.read_sql("Select * from expanded_roberta_prompts", conn)
conn.close()

## Check data

In [4]:
users_per_gender = all_prompts.groupby('gender')['user_id'].nunique().reset_index(name='num_users')
users_per_gender

,gender,num_users
0,Man (cisgender),15
1,Woman (cisgender),12


In [5]:
messages_per_user = all_prompts.groupby('user_id')['message_id'].nunique().reset_index(name='num_messages')
messages_per_user

,user_id,num_messages
0,6,9
1,8,2
2,11,11
3,15,3
4,16,25
5,25,4
6,28,22
7,31,5
8,34,66
9,46,5


## Set up Model

In [16]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

with open("../../prediction/finetune/label2id.json", "r") as f:
    label2id = json.load(f)

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = len(label2id)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(
        examples["conversational"],
        truncation=True,
        padding=False # padding is handled in the data collator
    )

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Check max sample size

In [17]:
texts = all_prompts['conversational'].tolist()

token_counts = [len(tokenizer.encode(text, add_special_tokens=True)) for text in texts]

max_tokens = max(token_counts)
min_tokens = min(token_counts)
avg_tokens = sum(token_counts) / len(token_counts)

print(f"Max tokens: {max_tokens}")
print(f"Min tokens: {min_tokens}")


Max tokens: 407
Min tokens: 4


## Cross Validation

- selected hyperparameters: lr 3.2e-5, batchsizes 8, epochs 5

In [18]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from sklearn.model_selection import GroupKFold

groups = all_prompts['user_id'].values
texts = all_prompts['conversational'].tolist()
labels = all_prompts['label'].tolist()
n_splits = 5  # e.g. 5-fold CV

gkf = GroupKFold(n_splits=n_splits)

all_results = []

for fold, (train_idx, val_idx) in enumerate(gkf.split(texts, labels, groups)):
    print(f"Fold {fold + 1}")

    train_prompts = all_prompts.iloc[train_idx]
    val_prompts = all_prompts.iloc[val_idx]

    train_dataset = Dataset.from_pandas(train_prompts[['conversational', 'label']])
    val_dataset = Dataset.from_pandas(val_prompts[['conversational', 'label']])

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)

    # Set up Trainer with model/tokenizer/data_collator as before
    trainer = Trainer(
        model_init=model_init,
        args=TrainingArguments(
            output_dir=f"./finetune/cross_validation/run_4/fold_{fold+1}_results",
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_dir=f"./fold_{fold+1}_logs",
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=5,
            learning_rate=3.2e-5,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            logging_steps=50,
            logging_strategy="steps",
        ),
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_metrics = trainer.evaluate()
    print(f"Fold {fold + 1} metrics:", eval_metrics)
    all_results.append(eval_metrics)

print(all_results)




Fold 1


Map: 100%|██████████| 114/114 [00:00<00:00, 32898.77 examples/s]
/var/folders/_h/dm14sczn7s77lrr765t54y7h0000gn/T/ipykernel_60864/18244806.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pred

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.633500,1.396640,0.324561,0.468703,0.982900,0.324561
2,0.436100,1.646291,0.350877,0.498677,0.982918,0.350877
3,0.313000,2.516136,0.456140,0.611588,0.963846,0.456140
4,0.134100,2.321332,0.561404,0.704024,0.967456,0.561404
5,0.149900,3.167446,0.438596,0.594857,0.963081,0.438596


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pin

Fold 1 metrics: {'eval_loss': 2.3213324546813965, 'eval_accuracy': 0.5614035087719298, 'eval_f1': 0.7040240461293094, 'eval_precision': 0.9674561403508772, 'eval_recall': 0.5614035087719298, 'eval_runtime': 0.4168, 'eval_samples_per_second': 273.492, 'eval_steps_per_second': 35.986, 'epoch': 5.0}
Fold 2


...

Fold 4 metrics: {'eval_loss': 0.8674131035804749, 'eval_accuracy': 0.7807017543859649, 'eval_f1': 0.7768381489311722, 'eval_precision': 0.8015873015873016, 'eval_recall': 0.7807017543859649, 'eval_runtime': 0.3534, 'eval_samples_per_second': 322.549, 'eval_steps_per_second': 42.441, 'epoch': 5.0}
Fold 5


Map: 100%|██████████| 112/112 [00:00<00:00, 30826.30 examples/s]
/var/folders/_h/dm14sczn7s77lrr765t54y7h0000gn/T/ipykernel_60864/18244806.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pred

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.642000,1.383812,0.196429,0.276603,0.906995,0.196429
2,0.502100,1.107049,0.526786,0.655293,0.909306,0.526786
3,0.296400,1.641585,0.517857,0.647657,0.908464,0.517857
4,0.329300,3.343460,0.401786,0.539513,0.894404,0.401786
5,0.086600,3.331898,0.428571,0.566042,0.898283,0.428571


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pin

Fold 5 metrics: {'eval_loss': 1.1070492267608643, 'eval_accuracy': 0.5267857142857143, 'eval_f1': 0.6552927978629208, 'eval_precision': 0.9093063186813186, 'eval_recall': 0.5267857142857142, 'eval_runtime': 0.4726, 'eval_samples_per_second': 236.99, 'eval_steps_per_second': 29.624, 'epoch': 5.0}
[{'eval_loss': 2.3213324546813965, 'eval_accuracy': 0.5614035087719298, 'eval_f1': 0.7040240461293094, 'eval_precision': 0.9674561403508772, 'eval_recall': 0.5614035087719298, 'eval_runtime': 0.4168, 'eval_samples_per_second': 273.492, 'eval_steps_per_second': 35.986, 'epoch': 5.0}, {'eval_loss': 0.6320773363113403, 'eval_accuracy': 0.672566371681416, 'eval_f1': 0.6237607411202007, 'eval_precision': 0.6120305456588643, 'eval_recall': 0.672566371681416, 'eval_runtime': 0.4662, 'eval_samples_per_second': 242.407, 'eval_steps_per_second': 32.178, 'epoch': 5.0}, {'eval_loss': 1.1333969831466675, 'eval_accuracy': 0.5789473684210527, 'eval_f1': 0.6079093902460049, 'eval_precision': 0.7136051335234638

In [19]:
results = pd.DataFrame(all_results)
stats = results.describe()
with open("stats.tex", "w") as f:
    stats.to_latex(f)


In [20]:
stats

,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.0
mean,1.212254,0.624081,0.673565,0.800797,0.624081,0.414500,277.81080,36.298800,5.0
std,0.652469,0.102850,0.068407,0.143862,0.102850,0.055651,39.43963,5.568248,0.0
min,0.632077,0.526786,0.607909,0.612031,0.526786,0.353400,236.99000,29.624000,5.0
25%,0.867413,0.561404,0.623761,0.713605,0.561404,0.363500,242.40700,32.178000,5.0
50%,1.107049,0.578947,0.655293,0.801587,0.578947,0.416800,273.49200,35.986000,5.0
75%,1.133397,0.672566,0.704024,0.909306,0.672566,0.466200,313.61600,41.265000,5.0
max,2.321332,0.780702,0.776838,0.967456,0.780702,0.472600,322.54900,42.441000,5.0


In [22]:
trainer.save_model("finetune/best_model")

## Get Validation set from best fold (4th)

In [10]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from sklearn.model_selection import GroupKFold

groups = all_prompts['user_id'].values
texts = all_prompts['conversational'].tolist()
labels = all_prompts['label'].tolist()
n_splits = 5

gkf = GroupKFold(n_splits=n_splits)

# Get validation indices for the 4th fold (fold==3)
for fold, (train_idx, val_idx) in enumerate(gkf.split(texts, labels, groups)):
    if fold == 3:
        val_prompts_4th = all_prompts.iloc[val_idx]
        val_dataset_4th = val_prompts_4th[['conversational', 'label']]
        break
val_dataset_4th


,conversational,label
48,what is the best way to encode and compress a ...,0
49,"does lzstring also work in the browser, client...",0
50,Is there a way in typescript to cast to a type...,0
51,I would like to distill a type based on an inc...,0
150,can you create Photoshop Scripts?,0
...,...,...
412,can you update directly the folder iterative c...,1
413,"super, now can you give me latex formula for t...",1
414,yes latex code please,1
515,I am working on the problem of reconstruc...,0


## save to db

In [13]:
conn  = sqlite3.connect('../../../data/giicg.db')
val_dataset_4th.to_sql("validation_set", conn, if_exists="replace", index=False)

114